# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
#I-1. Loop thru all subfolders of "event_data" and look for files to process

# Get current folder and subfolder event data and use it as filepath
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


In [3]:
#I-2 Process all files to create the data file "event_datafile_new.csv" that is used for Apache Casssandra tables

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# loop thru every filepath in the file path list 
for f in file_path_list:

    # read every csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        # fetch data one by one and fill up the "full_data_rows_list"        
        for line in csvreader:
            full_data_rows_list.append(line) 

# show total number of rows processed 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# Create file "event_datafile_new.csv"
# - define encoding and above registered dialect of csv file
# - create header row
# - insert data from "full_data_rows_list" to fill up the file via for loop
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
#I-3. Number of rows in newly created csv file "event_datafile_new.csv"
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. ETL Pipeline for Creating and Querying NoSQL Database

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the
<font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

![](images/image_event_datafile_new.jpg "Content of file image_event_datafile_new.csv")

#### Create a connection and keyspace

In [5]:
#II-1 Creation of a cluster and a connection to the database
from cassandra.cluster import Cluster
cluster = Cluster()

# Establish a connection and begin executing queries with a working session
session = cluster.connect()

#### Create Keyspace

In [6]:
# II-2 create keyspace "sparkify"

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#II-3 Set "sparkify" as current keyspace

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Prepare Database - Clean up all old tables if exist

In [8]:
## Drop all tables to clean up the database
drop_query1 = "DROP TABLE IF EXISTS ml_Q1_song_details_by_sessionid_itemInSession"
drop_query2 = "DROP TABLE IF EXISTS ml_Q2_song_details_by_userid_sessionid"
drop_query3 = "DROP TABLE IF EXISTS ml_Q3_user_by_song_listened"

# combine all queries in one list
drop_table_queries = [drop_query1, drop_query2, drop_query3]

# drop all tables if exists
try:
    for query in drop_table_queries:
        session.execute(query)
        
except Exception as e:
    print(e)        

# Create denormalized tables

## Query Example 1:
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and
itemInSession = 4

### CREATE table
Song details by sessionid and itemInSession

In [9]:
# create table ml_Q1_song_details_by_sessionid_itemInSession
query = "CREATE TABLE IF NOT EXISTS ml_Q1_song_details_by_sessionid_itemInSession "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY((sessionId, " \
                "itemInSession)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### INSERT data into table

In [10]:
# file with all collected data sets
file = 'event_datafile_new.csv'

# Open file, process all entries without header row and insert into table ml_Q1_song_details_by_sessionid_itemInSession
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO ml_Q1_song_details_by_sessionid_itemInSession (sessionId, itemInSession, artist, song" \
                ", length)"
        query = query + "values (%s, %s, %s, %s, %s);"

        ## execute INSERT statement with data from data file with given columns
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
            

### SELECT data and print out results

In [11]:
# create query
query = "select artist, song, length from ml_Q1_song_details_by_sessionid_itemInSession where sessionid = 338 and " \
        "iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


## print all results in a human readable format
result_data = []

# loop thru all collected rows and move it into a dataFrame
for row in rows:
    result_data.append(row)
    df = pd.DataFrame(result_data)

# print out data
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


---

## Query Example 2:
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for
userid = 10, sessionid = 182

### CREATE table
Song details by userid and sessionid

In [12]:
# create table ml_Q2_song_details_by_userid_sessionid
query = "CREATE TABLE IF NOT EXISTS ml_Q2_song_details_by_userid_sessionid "
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, user text, PRIMARY KEY((userid, " \
                "sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"
try:
    session.execute(query)
except Exception as e:
    print(e)

### INSERT data into table

In [13]:
# file with all collected data sets
file = 'event_datafile_new.csv'

# Open file, process all entries and insert into table ml_Q2_song_details_by_userid_sessionid
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO ml_Q2_song_details_by_userid_sessionid (userid, sessionId, itemInSession, artist, song" \
                ", user)"
        query = query + "values (%s, %s, %s, %s, %s, %s);"

    ## execute INSERT statement with data from data file with given columns
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

### SELECT data and print out results

In [14]:
# create query
query = "SELECT artist, song, user from ml_Q2_song_details_by_userid_sessionid WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## print all results in a human readable format
result_data = []

# loop thru all collected rows and move it into a dataFrame
for row in rows:
    result_data.append(row)
    df = pd.DataFrame(result_data)

# print out data
df

,artist,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


---

## Query Example 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### CREATE table
User by Song listened

In [15]:
# create table ml_Q3_user_by_song_listened
query = "CREATE TABLE IF NOT EXISTS ml_Q3_user_by_song_listened "
query = query + "(song text, userid int, user text, PRIMARY KEY ((song), userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)                            


### INSERT data into table

In [16]:
# file with all collected data sets
file = 'event_datafile_new.csv'

# Open file, process all entries and insert into table ml_Q3_user_by_song_listened
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO ml_Q3_user_by_song_listened (song, userid, user)"
        query = query + "values (%s, %s, %s);"

    ## execute INSERT statement with data from data file with given columns
        session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))

### SELECT data and print out results

In [17]:
# create query
query = "SELECT song, user from ml_Q3_user_by_song_listened WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## print all results in a human readable format
result_data = []

# loop thru all collected rows and move it into a dataFrame
for row in rows:
    result_data.append(row)
    df = pd.DataFrame(result_data)

# print out data
df

,song,user
0,All Hands Against His Own,Jacqueline Lynch
1,All Hands Against His Own,Tegan Levine
2,All Hands Against His Own,Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## Drop all tables to clean up the database
drop_query1 = "DROP TABLE IF EXISTS ml_Q1_song_details_by_sessionid_itemInSession"
drop_query2 = "DROP TABLE IF EXISTS ml_Q2_song_details_by_userid_sessionid"
drop_query3 = "DROP TABLE IF EXISTS ml_Q3_user_by_song_listened"

# combine all queries in one list
drop_table_queries = [drop_query1, drop_query2, drop_query3]

# drop all tables
try:
    for query in drop_table_queries:
        session.execute(query)
        
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()
